In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
from pprint import pp
import re
import json

In [2]:
data = pd.read_csv('../data/data.csv', sep=r'\t')

C:\Users\droic\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
data.shape

(39775, 172)

In [128]:
data.head()

,Q1A,Q1I,Q1E,Q2A,Q2I,Q2E,Q3A,Q3I,Q3E,Q4A,...,screensize,uniquenetworklocation,hand,religion,orientation,race,voted,married,familysize,major
0,4,28,3890,4,25,2122,2,16,1944,4,...,1,1,1,12,1,10,2,1,2,unknown
1,4,2,8118,1,36,2890,2,35,4777,3,...,2,1,2,7,0,70,2,1,4,unknown
2,3,7,5784,1,33,4373,4,41,3242,1,...,2,1,1,4,3,60,1,1,3,unknown
3,2,23,5081,3,11,6837,2,37,5521,1,...,2,1,2,4,5,70,2,1,5,biology
4,2,36,3215,2,13,7731,3,5,4156,4,...,2,2,3,10,1,10,2,1,4,Psychology


In [5]:
# profile = ProfileReport(data, title="Pandas Profiling Report", explorative=True)
# profile.to_file("your_report.html")

In [4]:
data.columns[data.isna().any()].tolist()

['country', 'major']

In [5]:
# fill those nan values with "unknown"
data[['country', 'major']] = data[['country', 'major']].fillna('unknown')

In [6]:
print(data.columns.to_list())

['Q1A', 'Q1I', 'Q1E', 'Q2A', 'Q2I', 'Q2E', 'Q3A', 'Q3I', 'Q3E', 'Q4A', 'Q4I', 'Q4E', 'Q5A', 'Q5I', 'Q5E', 'Q6A', 'Q6I', 'Q6E', 'Q7A', 'Q7I', 'Q7E', 'Q8A', 'Q8I', 'Q8E', 'Q9A', 'Q9I', 'Q9E', 'Q10A', 'Q10I', 'Q10E', 'Q11A', 'Q11I', 'Q11E', 'Q12A', 'Q12I', 'Q12E', 'Q13A', 'Q13I', 'Q13E', 'Q14A', 'Q14I', 'Q14E', 'Q15A', 'Q15I', 'Q15E', 'Q16A', 'Q16I', 'Q16E', 'Q17A', 'Q17I', 'Q17E', 'Q18A', 'Q18I', 'Q18E', 'Q19A', 'Q19I', 'Q19E', 'Q20A', 'Q20I', 'Q20E', 'Q21A', 'Q21I', 'Q21E', 'Q22A', 'Q22I', 'Q22E', 'Q23A', 'Q23I', 'Q23E', 'Q24A', 'Q24I', 'Q24E', 'Q25A', 'Q25I', 'Q25E', 'Q26A', 'Q26I', 'Q26E', 'Q27A', 'Q27I', 'Q27E', 'Q28A', 'Q28I', 'Q28E', 'Q29A', 'Q29I', 'Q29E', 'Q30A', 'Q30I', 'Q30E', 'Q31A', 'Q31I', 'Q31E', 'Q32A', 'Q32I', 'Q32E', 'Q33A', 'Q33I', 'Q33E', 'Q34A', 'Q34I', 'Q34E', 'Q35A', 'Q35I', 'Q35E', 'Q36A', 'Q36I', 'Q36E', 'Q37A', 'Q37I', 'Q37E', 'Q38A', 'Q38I', 'Q38E', 'Q39A', 'Q39I', 'Q39E', 'Q40A', 'Q40I', 'Q40E', 'Q41A', 'Q41I', 'Q41E', 'Q42A', 'Q42I', 'Q42E', 'country', 'source

### Make Dictionary from Codebook and save to json

In [7]:
with open('../data/codebook.txt', 'r') as f:
    codebook = f.read()
    lines = codebook.split('\n')

In [8]:
q_dict = {'questions': {}, 'answers': {}, 'TIPI_types': {}, 
          'TIPI_responses': {}, 'VCL': {}, 'other_questions': {}, 'other_answers': {}}
for line in lines:
    # get questions
    if re.search('^Q\d{1,2}\s', line):   
        QnA = line.split('\t', 1)
        q_dict['questions'][QnA[0]] = QnA[1].strip()
        
    # get answers
    if re.search('^\d = .*[aA]ppl.*', line):
        ans = line.split(' = ')
        q_dict['answers'][ans[0]] = ans[1].strip()
        
    # get TIPI (Ten Item Personality Inventory) types
    if re.search('^TIPI\d{1,2}\s', line):
        tipi = line.split('\t', 1)
        q_dict['TIPI_types'][tipi[0]] = tipi[1].strip()
        
    # get the TIPI responses
    if re.search('^\d = .*[aA]gree.*', line): 
        tipi = line.split(' = ')
        q_dict['TIPI_responses'][tipi[0]] = tipi[1].strip()
        
    # get VCL (Vocabulary) codes
    if re.search('^VCL', line):
        vcl = line.split('\t')
        q_dict['VCL'][vcl[0]] = vcl[1].strip()
    # get other questions
    if re.search('.*\s\".*\",\s\d', line):
        other = line.split('\t')
        q = line.split(r'"')[1].strip()
        q_dict['other_questions'][other[0]] = q
    # get other answers
        responses = line.split(r'"')[2].split(', ')[1:]
        q_dict['other_answers'][other[0]] = {resp.split('=')[0]: resp.split('=')[1].strip() 
                                             for resp in responses if '=' in resp}

            
    

In [9]:
with open('../data/codebook_dict.json', 'w') as f:
    json.dump(q_dict, f)

DASS results

Calculate DASS score and categories according to:
https://www.psytoolkit.org/survey-library/depression-anxiety-stress-dass.html

Severities of depression, anxiety and stress are categorized to:

    0 - Normal
    1 - Mild
    2 - Moderate
    3 - Severe
    4 - Extremely severe



In [10]:
df = data.copy()

DASS_keys = {'Depression': [3, 5, 10, 13, 16, 17, 21, 24, 26, 31, 34, 37, 38, 42],
             'Anxiety': [2, 4, 7, 9, 15, 19, 20, 23, 25, 28, 30, 36, 40, 41],
             'Stress': [1, 6, 8, 11, 12, 14, 18, 22, 27, 29, 32, 33, 35, 39]}

DASS_bins = {'Depression': [(0, 10), (10, 14), (14, 21), (21, 28)],
             'Anxiety': [(0, 8), (8, 10), (10, 15), (15, 20)],
             'Stress': [(0, 15), (15, 19), (19, 26), (26, 34)]}
             

for name, keys in DASS_keys.items():
    # Subtract one to match definition of DASS score in source
    df[name] = (df.filter(regex='Q(%s)A' % '|'.join(map(str, keys))) - 1).sum(axis=1)
    
    bins = DASS_bins[name]
    bins.append( (DASS_bins[name][-1][-1], df[name].max() + 1) )
    bins = pd.IntervalIndex.from_tuples(bins, closed='left')
    df[name + '_cat'] = np.arange(len(bins))[pd.cut(df[name], bins=bins).cat.codes]
    
dass = df[DASS_keys.keys()]
dass_cat = df[[k + '_cat' for k in DASS_keys.keys()]]



In [11]:
# Calculation of the big five personality attributes according to:
# http://gosling.psy.utexas.edu/wp-content/uploads/2014/09/JRP-03-tipi.pdfc

# Add personality types to data
personality_types = ['Extraversion', 'Agreeableness', 'Conscientiousness', 'EmotionalStability', 'Openness']

# Invert some entries
tipi = df.filter(regex='TIPI\d+').copy()
tipi_inv = tipi.filter(regex='TIPI(2|4|6|8|10)').apply(lambda d: 7 - d)
tipi[tipi.columns.intersection(tipi_inv.columns)] = tipi_inv

# Calculate scores
for idx, pt in enumerate( personality_types ):
    df[pt] = tipi[['TIPI{}'.format(idx + 1), 'TIPI{}'.format(6 + idx)]].mean(axis=1)

personalities = df[personality_types]

In [12]:
# There are no nan values
df.isna().sum().sum()

0

In [13]:
CMAP = 'Oranges'
df.describe().T.style.background_gradient(cmap = CMAP)

,count,mean,std,min,25%,50%,75%,max
Q1A,39775.000000,2.619485,1.032117,1.000000,2.000000,3.000000,4.000000,4.000000
Q1I,39775.000000,21.555977,12.133621,1.000000,11.000000,22.000000,32.000000,42.000000
Q1E,39775.000000,6970.590798,86705.131996,180.000000,2664.000000,3609.000000,5358.000000,12102282.000000
Q2A,39775.000000,2.172269,1.111563,1.000000,1.000000,2.000000,3.000000,4.000000
Q2I,39775.000000,21.248070,12.125288,1.000000,11.000000,21.000000,32.000000,42.000000
Q2E,39775.000000,5332.375839,26513.611044,176.000000,2477.000000,3511.000000,5216.000000,2161057.000000
Q3A,39775.000000,2.226097,1.038526,1.000000,1.000000,2.000000,3.000000,4.000000
Q3I,39775.000000,21.583004,12.115637,1.000000,11.000000,22.000000,32.000000,42.000000
Q3E,39775.000000,7426.446084,158702.352874,-10814.000000,2857.000000,3898.000000,5766.000000,28582689.000000
Q4A,39775.000000,1.950170,1.042218,1.000000,1.000000,2.000000,3.000000,4.000000


In [151]:
df

,Q1A,Q1I,Q1E,Q2A,Q2I,Q2E,Q3A,Q3I,Q3E,Q4A,...,Depression_cat,Anxiety,Anxiety_cat,Stress,Stress_cat,Extraversion,Agreeableness,Conscientiousness,EmotionalStability,Openness
0,4,28,3890,4,25,2122,2,16,1944,4,...,3,34,4,40,4,0.5,4.5,4.5,0.5,6.5
1,4,2,8118,1,36,2890,2,35,4777,3,...,3,17,3,27,3,4.5,4.5,2.0,0.5,3.5
2,3,7,5784,1,33,4373,4,41,3242,1,...,4,12,2,17,1,1.5,3.5,2.0,4.0,5.0
3,2,23,5081,3,11,6837,2,37,5521,1,...,2,17,3,16,1,2.0,6.0,6.5,4.5,6.0
4,2,36,3215,2,13,7731,3,5,4156,4,...,4,40,4,29,3,2.0,3.5,2.0,2.0,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39770,2,31,3287,1,5,2216,3,29,3895,2,...,4,12,2,15,1,2.0,5.0,3.5,2.5,3.5
39771,3,14,4792,4,41,2604,3,15,2668,4,...,4,34,4,34,4,2.5,3.0,2.5,2.0,3.5
39772,2,1,25147,1,4,4555,2,14,3388,1,...,0,1,0,5,0,5.0,3.5,6.5,3.5,4.5
39773,3,36,4286,1,34,2736,2,10,5968,2,...,2,13,2,33,3,1.5,2.0,3.5,1.5,3.0


In [14]:
df.to_csv('../data/data_labeled.csv')